## Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pygmid import Lookup as lk

## Load the LUT

In [2]:
n = lk('../nfet_03v3.mat')

## Specifications

In [3]:
gm = 10e-3

## Design Choices

In [4]:
L1 = 0.28 # um
gm_id = np.array([5, 10, 20])

## Sizing and benchmarking

In [5]:
id = gm/gm_id
jd = n.lookup('ID_W', GM_ID=gm_id, L=L1)
w = id/jd
cgg_w = n.lookup('CGG_W', GM_ID=gm_id, L=L1)
cgg = cgg_w*w
ft = gm/cgg/2/np.pi

In [6]:
df = pd.DataFrame([gm_id, id, jd, w, cgg, ft],['gm_id','id','jd','w','cgg','ft'],columns=['option1','option2','option3'])

In [7]:
df

,option1,option2,option3
gm_id,5.000000e+00,1.000000e+01,2.000000e+01
id,2.000000e-03,1.000000e-03,5.000000e-04
jd,2.539270e-05,6.847242e-06,4.903808e-07
w,7.876281e+01,1.460442e+02,1.019616e+03
cgg,9.257212e-14,1.603948e-13,9.581458e-13
ft,1.719253e+10,9.922699e+09,1.661072e+09


## Spice Validation (option 2)

In [8]:
%%writefile ./.spiceinit
set ngbehavior=hsa
set ng_nomodcheck
set color0=white
set color1=black
set xbrushwidth=2
set altshow

Overwriting ./.spiceinit


In [9]:
%%writefile ./sizing_diffpair.ngspice
** differential pair sizing example

.lib /foss/pdks/gf180mcuD/libs.tech/ngspice/sm141064.ngspice typical
.inc /foss/pdks/gf180mcuD/libs.tech/ngspice/design.ngspice
.param lx=0.28e-6 wx=146e-6 nfx=30 idx=1e-3

XM1 d g s 0 nfet_03v3 L={lx} W={wx} nf={nfx} 
+ ad='int((nf+2)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' 
+ pd='2*int((nf+2)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * W/nf * 0.18u' 
+ nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
XM2 d g s 0 nfet_03v3 L={lx} W={wx} nf={nfx} 
+ ad='int((nf+2)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' 
+ pd='2*int((nf+2)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * W/nf * 0.18u' 
+ nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1

vg g 0 1
vd d 0 2
ibias s 0 {2*idx}

.control
op
*show
let gm = @m.xm1.m0[gm]
let id = @m.xm1.m0[id]
let gm_id = @m.xm1.m0[gm]/@m.xm1.m0[id]
let cgg = @m.xm1.m0[cgg] + @m.xm1.m0[cgso] + @m.xm1.m0[cgdo]
print gm
print id
print cgg
.endc

Overwriting ./sizing_diffpair.ngspice


In [10]:
!/foss/tools/bin/ngspice -b ./sizing_diffpair.ngspice


Note: Compatibility modes selected: hs a



Circuit: ** differential pair sizing example

Doing analysis at TEMP = 27.000000 and TNOM = 27.000000

Using SPARSE 1.3 as Direct Linear Solver

No. of Data Rows : 1
gm = 9.983304e-03
id = 1.000001e-03
cgg = 1.596767e-13
Note: Simulation executed from .control section 
